https://docs.julialang.org/en/release-0.4/manual/unicode-input/

## DTW


In [13]:
δ(a,b) = (a - b)^2
# first(x) = x[1] firstは元からあるのでいらない
second(x) = x[2]

function minVal(v₁, v₂, v₃)
#    if first(v₁) ≦ minimum([first(v₂), first(v₃)])
    if first(v₁) <= minimum([first(v₂), first(v₃)])
        return v₁, 1
    elseif first(v₂) <= first(v₃)
        return v₂, 2
    else
        return v₃, 3
    end
end

function DTW(A, B)
    S = length(A)
    T = length(B)
    m = Matrix(S, T)
    m[1, 1] = [δ(A[1], B[1]), (0,0)]
    for i in 2:S
        m[i,1] = [m[i-1, 1][1] + δ(A[i], B[1]), [i-1, 1]]
    end
    for j in 2:T
        m[1,j] = [m[1, j-1][1] + δ(A[1], B[j]), [1, j-1]]
    end
    for i in 2:S
        for j in 2:T
            min, index = minVal(m[i-1,j], m[i,j-1], m[i-1,j-1])
            indexes = [[i-1, j], [i, j-1], [i-1, j-1]]
            m[i,j] = first(min) + δ(A[i],B[j]), indexes[index]
        end
    end
    return m
end

DTW (generic function with 1 method)

In [24]:
T = 150
A = sin((1:T)/10)
B = sin((1:T)/10 + pi*0.4)
C = [0 for i in 1:T]

150-element Array{Int64,1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [46]:
m = DTW(A, B)[150,150]

(9.3585894670215,[150,149])

In [26]:
DTW(A, C)[150,150][1]

77.67327710659531

## Stochastic Block Model

In [3]:
# Pkg.add("Einsum")
using Einsum

In [10]:
K = 8
α = 6
a₀ = b₀ = 0.5

m(z) = sum(z, 1)
α₁ = α₂ = ones(K,K) * α

function onehot(i, K)
    ret = zeros(K)
    ret[i] = 1
    return ret
end

function 𝕀(z)
    return Matrix([onehot(zᵢ) for zᵢ in z])
end

𝕀 (generic function with 1 method)

In [15]:
A = Matrix(2,2)
size(A)

(2,2)

In [7]:
𝕀 # \BbbI 

LoadError: UndefVarError: 𝕀 not defined

In [15]:
function update_z₁(X, z₁, z₂, i)
    N₁, N₂ = size(X)
    m₁ = m(z₁)
    m₂ = m(z₂)
    
    @einsum n⁺[k,l] := X[i,j] * 𝕀(z1)[i,k] * 𝕀(z2)[j,l]
    @einsum n⁻[k,l] := (ones(X)[i,j] - X[i,j]) * 𝕀(z1)[i,k] * 𝕀(z2)[j,l]
    return n⁺, n⁻
end

update_z₁ (generic function with 1 method)

In [18]:
using DataFrames
using CSV

INFO: Precompiling module DataFrames.
INFO: Precompiling module CSV.


In [90]:
data = readtable("./bi_data.csv")
#head(data)
1

1

In [88]:
hcat(data.columns...)

120×120 DataArrays.DataArray{Int64,2}:
 0  0  1  0  0  0  0  1  0  1  1  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  1  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  1  0  1  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  1  0  0  0  0  0  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  1  0  0  0  0  0  0  0  0  0  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  1  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  1  1  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  